## Studying NER

In [1]:
import pandas as pd
pd.pandas.set_option('display.max_columns', None)
import numpy as np
import sklearn
from ner import ner
from presidio_analyzer import AnalyzerEngine, BatchAnalyzerEngine, RecognizerRegistry


In [21]:
import spacy
import spacy_transformers
spacy.load('en_core_web_lg')


In [22]:
df = pd.read_csv("dataset/table_order.csv")
pd.pandas.set_option('display.max_columns', None)
df

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,10100,S18_1749,30,136.00,3,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Classic Cars,1:10,Welly Diecast Productions,*,4823.0,103.125817,213.098062
1,10100,S18_2248,50,55.09,2,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Vintage Cars,1:24,Red Start Diecast,*,2728.0,15.949021,33.802497
2,10100,S18_4409,22,75.46,4,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Vintage Cars,1:32,Studio M Art Models,*,2685.0,21.882745,43.659252
3,10100,S24_3969,49,35.29,1,2003-01-06,2003-01-13,2003-01-10,Shipped,NaN,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,NaN,Nashua,NH,62005,USA,1216.0,114200,*,Vintage Cars,1:32,Autoart Studio Design,*,550.0,16.061205,35.118038
4,10101,S18_2325,25,108.06,4,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,NaN,Frankfurt,NaN,60528,Germany,1504.0,59700,*,Trucks and Buses,1:32,Autoart Studio Design,*,24.0,25.660152,46.427639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991,10425,S24_2300,49,127.79,9,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Vintage Cars,1:18,Welly Diecast Productions,*,6763.0,71.973408,117.440000
2992,10425,S24_2840,31,31.82,5,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Classic Cars,1:18,Gearbox Collectibles,*,9771.0,94.808700,152.250990
2993,10425,S32_1268,41,83.79,11,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Classic Cars,1:18,Min Lin Diecast,*,9762.0,69.061199,138.994773
2994,10425,S32_2509,11,50.32,6,2005-05-31,2005-06-07,NaN,In Process,NaN,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200,*,Classic Cars,1:18,Red Start Diecast,*,8909.0,93.836843,208.542691


In [23]:
df = df.sample(n=min(1000, df.shape[0])) 


In [24]:
analyzer = AnalyzerEngine(supported_languages=["en", "fr"])
batch_analyzer = BatchAnalyzerEngine(analyzer_engine=analyzer)

In [25]:
analyzer_results = list(batch_analyzer.analyze_dict(df.to_dict(orient="list"), language="en"))

In [26]:
analyzer_results

[DictAnalyzerResult(key='orderNumber', value=[10232, 10312, 10244, 10114, 10127, 10163, 10381, 10109, 10307, 10225, 10206, 10373, 10248, 10149, 10271, 10333, 10306, 10245, 10200, 10179, 10196, 10146, 10339, 10383, 10252, 10388, 10221, 10304, 10358, 10340, 10195, 10348, 10371, 10117, 10179, 10393, 10106, 10412, 10227, 10135, 10197, 10340, 10425, 10248, 10407, 10359, 10223, 10425, 10313, 10223, 10119, 10139, 10107, 10133, 10380, 10380, 10263, 10287, 10143, 10416, 10212, 10393, 10357, 10164, 10332, 10285, 10197, 10122, 10106, 10296, 10217, 10246, 10181, 10358, 10266, 10350, 10104, 10411, 10140, 10263, 10288, 10124, 10403, 10360, 10236, 10165, 10155, 10350, 10197, 10396, 10201, 10135, 10405, 10331, 10319, 10181, 10121, 10204, 10155, 10155, 10331, 10311, 10232, 10182, 10381, 10129, 10259, 10310, 10203, 10235, 10114, 10142, 10296, 10106, 10392, 10140, 10262, 10167, 10342, 10299, 10105, 10200, 10305, 10108, 10379, 10342, 10389, 10103, 10344, 10327, 10276, 10138, 10336, 10313, 10182, 10117, 10

In [44]:
#questo ciclo controlla per ogni colonna, se il suo tipo è object. Se è object, allora aggiunge al dizionario l 
#un'entrata la cui chiave sarà il nome della colonna e il cui valore è la lista delle entity  delle varie liste 
#presenti in recognizer_results solo se queste liste non sono vuote
l = {}
for w, j in zip(df.columns, analyzer_results):
        if df[w].dtype == 'object':
                l[w] = [i[0].entity_type for i in j.recognizer_results if len(i) > 0]

In [45]:
l

{'productCode': ['PERSON',
  'PERSON',
  'PERSON',
  'LOCATION',
  'NRP',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'NRP',
  'PERSON',
  'PERSON',
  'NRP',
  'PHONE_NUMBER',
  'PERSON',
  'PERSON',
  'PHONE_NUMBER',
  'PERSON',
  'PHONE_NUMBER',
  'PERSON',
  'PERSON',
  'NRP',
  'NRP',
  'LOCATION',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PHONE_NUMBER',
  'LOCATION',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PHONE_NUMBER',
  'LOCATION',
  'PERSON',
  'NRP',
  'NRP',
  'PERSON',
  'PERSON',
  'NRP',
  'PERSON',
  'PHONE_NUMBER',
  'PERSON',
  'PERSON',
  'LOCATION',
  'PERSON',
  'PERSON',
  'NRP',
  'NRP',
  'NRP',
  'LOCATION',
  'PERSON',
  'PHONE_NUMBER',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PERSON',
  'PHONE_NUMBER',
  'LOCATION',
  'PHONE_NUMBER',
  'PHONE_NUMBER',
  'PERSON',
  'LOCATION',
  'PERSON',
  'PHO

In [46]:
list_entities = {}
cols = [col for col in df.columns]

In [47]:
cols

['orderNumber',
 'productCode',
 'quantityOrdered',
 'priceEach',
 'orderLineNumber',
 'orderDate',
 'requiredDate',
 'shippedDate',
 'status',
 'comments',
 'customerNumber',
 'customerName',
 'contactLastName',
 'contactFirstName',
 'phone',
 'addressLine1',
 'addressLine2',
 'city',
 'state',
 'postalCode',
 'country',
 'salesRepEmployeeNumber',
 'creditLimit',
 'productName',
 'productLine',
 'productScale',
 'productVendor',
 'productDescription',
 'quantityInStock',
 'buyPrice',
 'MSRP']

In [48]:
for col in cols:
        list_entities[col] = None

In [49]:
#creo lista con solo le chiavi (colonne) la cui lista di valori contiene 
#un numero di elementi maggiore di 0.1 * il numero di rows nel campione
cols = [col for col in l.keys() if len(l[col]) > 0.1 * df.shape[0]]
cols

['orderDate',
 'requiredDate',
 'shippedDate',
 'comments',
 'customerName',
 'contactLastName',
 'contactFirstName',
 'phone',
 'addressLine1',
 'addressLine2',
 'city',
 'state',
 'postalCode',
 'country',
 'productName',
 'productLine',
 'productScale',
 'productVendor',
 'productDescription']

In [50]:
for col in cols:
    lst = [i for i in l[col]]
    if ('LOCATION' in lst) and ('name' not in col.lower()) and (
            len([i for i in l[col] if i == 'LOCATION']) > 0.1 * df.shape[0]):
            #il secondo elemento è una sorta di probabilità/confidenza che quella colonna sia di quel determinato tipo
        list_entities[col] = ['LOCATION', len([i for i in l[col] if i == 'LOCATION'])/df.shape[0]]
    else:
        most_freq = max(set(lst), key=lst.count)
        list_entities[col] = [most_freq, len([i for i in lst if i == most_freq])/df.shape[0]]

In [51]:
loc_cols = [[i, list_entities[i][0]] for i in list_entities.keys() if list_entities[i] is not None]
loc_cols

[['orderDate', 'DATE_TIME'],
 ['requiredDate', 'DATE_TIME'],
 ['shippedDate', 'DATE_TIME'],
 ['comments', 'LOCATION'],
 ['customerName', 'PERSON'],
 ['contactLastName', 'PERSON'],
 ['contactFirstName', 'PERSON'],
 ['phone', 'PHONE_NUMBER'],
 ['addressLine1', 'LOCATION'],
 ['addressLine2', 'LOCATION'],
 ['city', 'LOCATION'],
 ['state', 'LOCATION'],
 ['postalCode', 'DATE_TIME'],
 ['country', 'LOCATION'],
 ['productName', 'LOCATION'],
 ['productLine', 'LOCATION'],
 ['productScale', 'LOCATION'],
 ['productVendor', 'LOCATION'],
 ['productDescription', 'LOCATION']]

In [52]:
loc_cols = [i[0] for i in loc_cols if i[1] == 'LOCATION']
loc_cols

['comments',
 'addressLine1',
 'addressLine2',
 'city',
 'state',
 'country',
 'productName',
 'productLine',
 'productScale',
 'productVendor',
 'productDescription']

In [53]:
addresses = ['street', 'rue', 'via', 'square', 'avenue', 'place', 'strada']
for col in loc_cols:
    list_addresses = [i for i in df[col].fillna('none') if any(ele in i.lower() for ele in addresses)]
    if len(list_addresses) > 0.1 * int(df.shape[0]):
        list_entities[col] = ['ADDRESS', len(list_addresses)/int(df.shape[0])]


In [16]:
list_addresses

[]

In [17]:
list_entities #è un dizionario chiave/colonna: [entity_type, confidence]

{'orderNumber': None,
 'productCode': None,
 'quantityOrdered': None,
 'priceEach': None,
 'orderLineNumber': None,
 'orderDate': ['DATE_TIME', 1.0],
 'requiredDate': ['DATE_TIME', 1.0],
 'shippedDate': ['DATE_TIME', 0.947],
 'status': None,
 'comments': ['LOCATION', 0.745],
 'customerNumber': None,
 'customerName': ['PERSON', 0.076],
 'contactLastName': ['PERSON', 0.505],
 'contactFirstName': ['PERSON', 0.817],
 'phone': ['PHONE_NUMBER', 0.57],
 'addressLine1': ['ADDRESS', 0.24],
 'addressLine2': ['LOCATION', 0.808],
 'city': ['LOCATION', 0.783],
 'state': ['LOCATION', 0.69],
 'postalCode': ['DATE_TIME', 0.057],
 'country': ['LOCATION', 1.0],
 'salesRepEmployeeNumber': None,
 'creditLimit': None,
 'productName': ['LOCATION', 0.101],
 'productLine': ['LOCATION', 0.101],
 'productScale': ['LOCATION', 0.101],
 'productVendor': ['LOCATION', 0.101],
 'productDescription': ['LOCATION', 0.101],
 'quantityInStock': None,
 'buyPrice': None,
 'MSRP': None}

## Trying to improve ner function

In [223]:
from typing import List
import pprint

from presidio_analyzer import AnalyzerEngine, PatternRecognizer, EntityRecognizer, Pattern, RecognizerResult
from presidio_analyzer.recognizer_registry import RecognizerRegistry
from presidio_analyzer.nlp_engine import NlpEngine, SpacyNlpEngine, NlpArtifacts, NlpEngineProvider
from presidio_analyzer.context_aware_enhancers import LemmaContextAwareEnhancer

In [121]:
df = pd.read_csv("dataset/table_order.csv")


In [119]:
df = df.sample(n=min(1000, df.shape[0]))

In [99]:
df = df.fillna(0)

In [100]:
df

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
1906,10306,S12_3148,34,145.04,14,2004-10-14,2004-10-21,2004-10-17,Shipped,0,187,"AV Stores, Co.",Ashworth,Rachel,(171) 555-1555,Fauntleroy Circus,0,Manchester,0,EC2 5NT,UK,1501.0,136800,*,Classic Cars,1:10,Gearbox Collectibles,*,6627.0,103.123212,214.006810
1693,10282,S700_1938,43,77.95,2,2004-08-20,2004-08-26,2004-08-22,Shipped,0,124,Mini Gifts Distributors Ltd.,Nelson,Susan,4155551450,5677 Strong St.,0,San Rafael,CA,97562,USA,1165.0,210500,*,Trucks and Buses,1:12,Red Start Diecast,*,1639.0,102.323096,212.143631
1738,10287,S12_4675,23,107.10,9,2004-08-30,2004-09-06,2004-09-01,Shipped,0,298,"Vida Sport, Ltd",Holz,Mihael,0897-034555,Grenzacherweg 237,0,Genève,0,1203,Switzerland,1702.0,141300,*,Trucks and Buses,1:18,Autoart Studio Design,*,167.0,66.875166,123.122110
1150,10222,S24_2841,32,56.86,5,2004-02-19,2004-02-27,2004-02-20,Shipped,0,239,Collectable Mini Designs Co.,Thompson,Valarie,7605558146,361 Furth Circle,0,San Diego,CA,91217,USA,1166.0,105000,*,Vintage Cars,1:18,Second Gear Diecast,*,3568.0,60.241250,76.512118
1417,10253,S18_2870,37,114.84,2,2004-06-01,2004-06-09,2004-06-02,Cancelled,Customer disputed the order and we agreed to c...,201,"UK Collectables, Ltd.",Devon,Elizabeth,(171) 555-2282,"12, Berkeley Gardens Blvd",0,Liverpool,0,WX1 6LT,UK,1501.0,92700,*,Trucks and Buses,1:12,Welly Diecast Productions,*,35.0,95.897261,201.351694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2379,10354,S10_4698,42,178.17,3,2004-12-04,2004-12-10,2004-12-05,Shipped,0,323,"Down Under Souveniers, Inc",Graham,Mike,+64 9 312 5555,162-164 Grafton Road,Level 2,Auckland,0,0,New Zealand,1612.0,88000,*,Trucks and Buses,1:32,Welly Diecast Productions,*,24.0,32.592150,55.370045
2089,10322,S10_4962,46,141.83,8,2004-11-04,2004-11-12,2004-11-10,Shipped,Customer has worked with some of our vendors i...,363,Online Diecast Creations Co.,Young,Dorothy,6035558647,2304 Long Airport Avenue,0,Nashua,NH,62005,USA,1216.0,114200,*,Vintage Cars,1:32,Carousel DieCast Legends,*,3389.0,15.918814,35.040411
2831,10404,S18_3482,28,127.88,5,2005-04-08,2005-04-14,2005-04-11,Shipped,0,323,"Down Under Souveniers, Inc",Graham,Mike,+64 9 312 5555,162-164 Grafton Road,Level 2,Auckland,0,0,New Zealand,1612.0,88000,*,Vintage Cars,1:24,Carousel DieCast Legends,*,6971.0,15.932137,33.359247
386,10143,S24_3420,33,59.83,9,2003-08-10,2003-08-18,2003-08-12,Shipped,Can we deliver the new Ford Mustang models by ...,320,Mini Creations Ltd.,Huang,Wing,5085559555,4575 Hillside Dr.,0,New Bedford,MA,50553,USA,1188.0,94500,*,Trucks and Buses,1:10,Motor City Art Classics,*,6618.0,103.022047,199.418676


In [101]:
addresses = ['Street', 'Rue', 'Via', 'Square', 'Avenue', 'Place', 'Strada', 'St', 'Lane', 
    'Road', 'Boulevard', 'Ln', "Rd", "Highway" "Drive", "Av", "Hwy", "Blvd", "Corso", "Piazza", 
    "Calle", "Plaza", "Avenida", "Rambla", "C/"]

In [102]:
addresses_recognizer = PatternRecognizer(supported_entity="ADDRESS", deny_list=addresses)


In [103]:
#using a roberta-based language model to improve performance
'''configuration = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_trf"}],
}

# Create NLP engine based on configuration
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine_robertaBased = provider.create_engine()

# Pass the created NLP engine and supported_languages to the AnalyzerEngine
analyzer = AnalyzerEngine(
    nlp_engine = nlp_engine_robertaBased, 
    supported_languages=["en", "fr", "es", "it"]
) '''

'configuration = {\n    "nlp_engine_name": "spacy",\n    "models": [{"lang_code": "en", "model_name": "en_core_web_trf"}],\n}\n\n# Create NLP engine based on configuration\nprovider = NlpEngineProvider(nlp_configuration=configuration)\nnlp_engine_robertaBased = provider.create_engine()\n\n# Pass the created NLP engine and supported_languages to the AnalyzerEngine\nanalyzer = AnalyzerEngine(\n    nlp_engine = nlp_engine_robertaBased, \n    supported_languages=["en", "fr", "es", "it"]\n) '

In [104]:
analyzer = AnalyzerEngine(supported_languages=["en", "fr"])
analyzer.registry.add_recognizer(addresses_recognizer)
batch_analyzer = BatchAnalyzerEngine(analyzer_engine=analyzer)

In [105]:
analyzer_results = list(batch_analyzer.analyze_dict(df.to_dict(orient="list"), language="en"))

In [106]:
analyzer_results

[DictAnalyzerResult(key='orderNumber', value=[10306, 10282, 10287, 10222, 10253, 10276, 10266, 10231, 10271, 10103, 10390, 10393, 10266, 10312, 10367, 10263, 10312, 10288, 10328, 10138, 10195, 10224, 10244, 10168, 10359, 10106, 10402, 10115, 10304, 10350, 10285, 10105, 10375, 10375, 10278, 10336, 10182, 10142, 10350, 10414, 10312, 10350, 10347, 10175, 10400, 10124, 10329, 10401, 10151, 10135, 10342, 10412, 10153, 10220, 10418, 10227, 10222, 10367, 10339, 10400, 10105, 10170, 10284, 10310, 10301, 10411, 10207, 10390, 10251, 10143, 10228, 10110, 10125, 10124, 10350, 10358, 10371, 10322, 10117, 10411, 10310, 10287, 10207, 10283, 10208, 10147, 10386, 10238, 10223, 10268, 10383, 10223, 10400, 10328, 10305, 10414, 10371, 10150, 10324, 10210, 10206, 10181, 10237, 10318, 10314, 10173, 10340, 10230, 10306, 10227, 10105, 10279, 10292, 10283, 10105, 10154, 10171, 10391, 10120, 10196, 10289, 10292, 10319, 10183, 10235, 10275, 10373, 10114, 10361, 10168, 10292, 10264, 10210, 10363, 10279, 10329, 10

In [107]:
df.columns

Index(['orderNumber', 'productCode', 'quantityOrdered', 'priceEach',
       'orderLineNumber', 'orderDate', 'requiredDate', 'shippedDate', 'status',
       'comments', 'customerNumber', 'customerName', 'contactLastName',
       'contactFirstName', 'phone', 'addressLine1', 'addressLine2', 'city',
       'state', 'postalCode', 'country', 'salesRepEmployeeNumber',
       'creditLimit', 'productName', 'productLine', 'productScale',
       'productVendor', 'productDescription', 'quantityInStock', 'buyPrice',
       'MSRP'],
      dtype='object')

In [108]:
list_entities = {}
df_cols = [col for col in df.columns]

In [109]:
for col in df_cols:
    list_entities[col] = None

In [110]:
for col in df_cols:
    #assigning ZIPCODE entity if zipcode in dataset is 'object' type
    if (('postal' in col.lower()) and ('code' in col.lower())) or (('zip' in col.lower()) and (
        'code' in col.lower())) or (('zip' in col.lower())):
        list_entities[col] = ['ZIPCODE', 1.0]
        #assigning CREDIT_CARD_NUMBER entity if credit card number in dataset is 'object' type
    elif (('credit' in col.lower()) or ('card' in col.lower())) and ('number' in col.lower()):
        list_entities[col] = ['CREDIT_CARD_NUMBER', 1.0]

In [111]:
dict_object_entity = {}
for w, j in zip(df.columns, analyzer_results):
    if df[w].dtype == 'object':
        dict_object_entity[w] = [i[0].entity_type for i in j.recognizer_results if len(i) > 0]

In [112]:
cols_object = [col for col in dict_object_entity.keys() if len(dict_object_entity[col]) > 0.1 * df.shape[0]]

In [113]:
cols_object

['orderDate',
 'requiredDate',
 'shippedDate',
 'customerName',
 'contactLastName',
 'contactFirstName',
 'phone',
 'addressLine1',
 'city',
 'state',
 'postalCode',
 'country',
 'productVendor']

In [114]:
for col in cols_object:
    lst = [i for i in dict_object_entity[col]]
    #assigning LOCATION entity
    if ('LOCATION' in lst) and ('name' not in col.lower()) and (
            len([i for i in dict_object_entity[col] if i == 'LOCATION']) > 0.1 * df.shape[0]):
        list_entities[col] = ['LOCATION', len([i for i in dict_object_entity[col] if i == 'LOCATION'])/df.shape[0]]
    #assigning ZIPCODE entity if zipcode in dataset is 'object' type
    elif (('postal' in col.lower()) and ('code' in col.lower())) or (('zip' in col.lower()) and (
            'code' in col.lower())) or (('zip' in col.lower())): 
        list_entities[col] = ['ZIPCODE', 1.0]
    #assigning CREDIT_CARD_NUMBER entity if credit card number in dataset is 'object' type
    elif ((('credit' in col.lower()) or ('card' in col.lower())) and ('number' in col.lower())):
        list_entities[col] = ['CREDIT_CARD_NUMBER', 1.0]
    else:
        most_freq = max(set(lst), key=lst.count)
        list_entities[col] = [most_freq, len([i for i in lst if i == most_freq])/df.shape[0]]

In [115]:
list_entities


{'orderNumber': None,
 'productCode': None,
 'quantityOrdered': None,
 'priceEach': None,
 'orderLineNumber': None,
 'orderDate': ['DATE_TIME', 1.0],
 'requiredDate': ['DATE_TIME', 1.0],
 'shippedDate': ['DATE_TIME', 0.951],
 'status': None,
 'comments': None,
 'customerNumber': None,
 'customerName': ['PERSON', 0.066],
 'contactLastName': ['PERSON', 0.508],
 'contactFirstName': ['PERSON', 0.821],
 'phone': ['PHONE_NUMBER', 0.596],
 'addressLine1': ['ADDRESS', 0.554],
 'addressLine2': None,
 'city': ['LOCATION', 0.773],
 'state': ['LOCATION', 0.144],
 'postalCode': ['ZIPCODE', 1.0],
 'country': ['LOCATION', 1.0],
 'salesRepEmployeeNumber': None,
 'creditLimit': None,
 'productName': None,
 'productLine': None,
 'productScale': None,
 'productVendor': ['PERSON', 0.145],
 'productDescription': None,
 'quantityInStock': None,
 'buyPrice': None,
 'MSRP': None}

## Test ner on other datasets

In [226]:
df_tableOrder = pd.read_csv("dataset/table_order.csv")

In [2]:
ner("dataset/table_order.csv")

Starting...


{'orderNumber': None,
 'productCode': None,
 'quantityOrdered': None,
 'priceEach': None,
 'orderLineNumber': None,
 'orderDate': ['DATE_TIME', 1.0],
 'requiredDate': ['DATE_TIME', 1.0],
 'shippedDate': ['DATE_TIME', 0.965],
 'status': None,
 'comments': None,
 'customerNumber': None,
 'customerName': ['PERSON', 0.072],
 'contactLastName': ['PERSON', 0.513],
 'contactFirstName': ['PERSON', 0.819],
 'phone': ['PHONE_NUMBER', 0.579],
 'addressLine1': ['ADDRESS', 0.567],
 'addressLine2': None,
 'city': ['LOCATION', 0.782],
 'state': ['LOCATION', 0.137],
 'postalCode': ['ZIPCODE', 1.0],
 'country': ['LOCATION', 1.0],
 'salesRepEmployeeNumber': None,
 'creditLimit': None,
 'productName': None,
 'productLine': None,
 'productScale': None,
 'productVendor': ['PERSON', 0.147],
 'productDescription': None,
 'quantityInStock': None,
 'buyPrice': None,
 'MSRP': None}

#### telco_customer dataset

In [3]:
df_ibm = pd.read_csv("dataset/Telco_customer_churn.csv")


In [4]:
df_ibm

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,1,United States,California,Landers,92285,"34.341737, -116.539416",34.341737,-116.539416,Female,No,No,No,72,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),21.15,1419.4,No,0,45,5306,NaN
7039,6840-RESVB,1,United States,California,Adelanto,92301,"34.667815, -117.536183",34.667815,-117.536183,Male,No,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No,0,59,2140,NaN
7040,2234-XADUH,1,United States,California,Amboy,92304,"34.559882, -115.637164",34.559882,-115.637164,Female,No,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No,0,71,5560,NaN
7041,4801-JZAZL,1,United States,California,Angelus Oaks,92305,"34.1678, -116.86433",34.167800,-116.864330,Female,No,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No,0,59,2793,NaN


In [6]:
df_ibm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [3]:
ner("dataset/Telco_customer_churn.csv")

Starting...


{'CustomerID': None,
 'Count': None,
 'Country': ['LOCATION', 1.0],
 'State': ['LOCATION', 1.0],
 'City': ['LOCATION', 0.522],
 'Zip Code': ['ZIPCODE', 1.0],
 'Lat Long': ['US_DRIVER_LICENSE', 0.923],
 'Latitude': None,
 'Longitude': None,
 'Gender': None,
 'Senior Citizen': None,
 'Partner': None,
 'Dependents': None,
 'Tenure Months': None,
 'Phone Service': None,
 'Multiple Lines': None,
 'Internet Service': None,
 'Online Security': None,
 'Online Backup': None,
 'Device Protection': None,
 'Tech Support': None,
 'Streaming TV': None,
 'Streaming Movies': None,
 'Contract': ['DATE_TIME', 1.0],
 'Paperless Billing': None,
 'Payment Method': None,
 'Monthly Charges': None,
 'Total Charges': None,
 'Churn Label': None,
 'Churn Value': None,
 'Churn Score': None,
 'CLTV': None,
 'Churn Reason': None}

#### List of real usa addresses

In [18]:
df_USAddresses = pd.read_csv("dataset/list_of_real_usa_addresses.csv")


In [9]:
df_USAddresses[0:50]

,address,city,state,zip
0,777 Brockton Avenue,Abington,MA,2351
1,30 Memorial Drive,Avon,MA,2322
2,250 Hartford Avenue,Bellingham,MA,2019
3,700 Oak Street,Brockton,MA,2301
4,66-4 Parkhurst Rd,Chelmsford,MA,1824
5,591 Memorial Dr,Chicopee,MA,1020
6,55 Brooksby Village Way,Danvers,MA,1923
7,137 Teaticket Hwy,East Falmouth,MA,2536
8,42 Fairhaven Commons Way,Fairhaven,MA,2719
9,374 William S Canning Blvd,Fall River,MA,2721


In [10]:
df_USAddresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   address  234 non-null    object
 1   city     234 non-null    object
 2   state    234 non-null    object
 3   zip      234 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 7.4+ KB


In [5]:
ner("dataset/list_of_real_usa_addresses.csv")

Starting...


{'address': ['ADDRESS', 0.6153846153846154],
 'city': ['LOCATION', 0.5384615384615384],
 'state': ['LOCATION', 0.6794871794871795],
 'zip': ['ZIPCODE', 1.0]}

#### list of credit number and names

In [5]:
df_creditNumber = pd.read_csv("dataset/1-MB-Test.csv")

In [6]:
df_creditNumber

,First and Last Name,SSN,Credit Card Number
0,NaN,NaN,Visa MC AMEX
1,Robert Aragon,489-36-8350,4929-3813-3266-4295
2,Ashley Borden,514-14-8905,5370-4638-8881-3020
3,Thomas Conley,690-05-5315,4916-4811-5814-8111
4,Susan Davis,421-37-1396,4916-4034-9269-8783
...,...,...,...
10551,Albert Iorio,322-84-2303,4916-6734-7572-5037
10552,Charles Jackson,646-44-9083,5218-0144-2703-9288
10553,Teresa Kaminski,465-73-5044,5399-0706-4128-0200
10554,Tim Lowe,044-34-6976,5144-8691-2776-1130


In [7]:
df_creditNumber = df_creditNumber.sample(n=min(1000, df_creditNumber.shape[0]))
df_creditNumber = df_creditNumber.fillna(0)

In [9]:
analyzer = AnalyzerEngine(supported_languages=["en", "fr", "it", "es"])
batch_analyzer = BatchAnalyzerEngine(analyzer_engine=analyzer)

In [11]:
analyzer_results = list(batch_analyzer.analyze_dict(df_creditNumber.to_dict(orient="list"), language="en"))

In [12]:
analyzer_results

[DictAnalyzerResult(key='First and Last Name', value=['Victor\xa0Faulkner', 'Ashley\xa0Borden', 'Lillian\xa0Venson', 'Lynette\xa0Oyola', 'Ashley\xa0Borden', 'Jacki\xa0Russell', 'Christopher\xa0Diaz', 'Monte\xa0Mceachern', 'Charles\xa0Jackson', 'Mireille\xa0Townsend', 'Teresa\xa0Kaminski', 'Robert\xa0Aragon', 'Charles\xa0Jackson', 'Jerome\xa0Munsch', 'Victor\xa0Faulkner', 'Rebecca\xa0Zwick', 'Danny\xa0Reyes', 'Danny\xa0Reyes', 'Rick\xa0Edwards', 'Charles\xa0Jackson', 'Thomas\xa0Santos', 'Adriane\xa0Morrison', 'Lynette\xa0Oyola', 'Gail\xa0Watson', 'Christopher\xa0Diaz', 'Johnson\xa0White', 'Agnes\xa0Nelson', 'Thomas\xa0Conley', 'Adriane\xa0Morrison', 'Ashley\xa0Borden', 'James\xa0Heard', 'Robert\xa0Aragon', 'Agnes\xa0Nelson', 'Mireille\xa0Townsend', 'Stacey\xa0Peacock', 'Victor\xa0Faulkner', 'Lynette\xa0Oyola', 'Monte\xa0Mceachern', 'Jacki\xa0Russell', 'Thomas\xa0Conley', 'Christopher\xa0Diaz', 'Mireille\xa0Townsend', 'Gail\xa0Watson', 'Mark\xa0Hall', 'James\xa0Heard', 'Gail\xa0Watson', 

In [13]:
l = {}
for w, j in zip(df_creditNumber.columns, analyzer_results):
    if df_creditNumber[w].dtype == 'object':
        l[w] = [i[0].entity_type for i in j.recognizer_results if len(i) > 0]

In [15]:
list_entities = {}
cols = [col for col in df_creditNumber.columns]

In [16]:
for col in cols:
    list_entities[col] = None

In [18]:
cols = [col for col in l.keys() if len(l[col]) > 0.1 * df_creditNumber.shape[0]]

In [19]:
cols

['First and Last Name', 'SSN', 'Credit Card Number']

In [32]:
for col in cols:
    lst = [i for i in l[col]]
    #assigning LOCATION entity
    if ('LOCATION' in lst) and ('name' not in col.lower()) and (
            len([i for i in l[col] if i == 'LOCATION']) > 0.1 * df_creditNumber.shape[0]):
        list_entities[col] = ['LOCATION', len([i for i in l[col] if i == 'LOCATION'])/df_creditNumber.shape[0]]
    #assigning ZIPCODE entity if zipcode in dataset is 'object' type
    elif (('postal' in col.lower()) and ('code' in col.lower())) or (('zip' in col.lower()) and (
            'code' in col.lower())) or (('zip' in col.lower())): 
        list_entities[col] = ['ZIPCODE', 1.0]
    elif ((('credit' in col.lower()) or ('card' in col.lower())) and ('number' in col.lower())):
        list_entities[col] = ['CREDIT_CARD_NUMBER', 1.0]
    else:
        most_freq = max(set(lst), key=lst.count)
        list_entities[col] = [most_freq, len([i for i in lst if i == most_freq])/df_creditNumber.shape[0]]

In [33]:
list_entities

{'First and Last Name': ['PERSON', 0.895],
 'SSN': ['US_SSN', 0.938],
 'Credit Card Number': ['CREDIT_CARD_NUMBER', 1.0]}

In [6]:
ner("dataset/1-MB-Test.csv")

Starting...


{'First and Last Name': ['PERSON', 0.897],
 'SSN': ['US_SSN', 0.932],
 'Credit Card Number': ['CREDIT_CARD_NUMBER', 1.0]}

#### Unwanted Calls

In [125]:
ner("dataset/Consumer_Complaints_Data_-_Unwanted_Calls.csv")

Starting...


{'Ticket ID': None,
 'Ticket Created': ['DATE_TIME', 0.954],
 'Date of Issue': ['DATE_TIME', 0.993],
 'Time of Issue': ['DATE_TIME', 0.72],
 'Form': None,
 'Method': None,
 'Issue': None,
 'Caller ID Number': ['PHONE_NUMBER', 0.663],
 'Type of Call or Messge': None,
 'Advertiser Business Number': ['PHONE_NUMBER', 0.195],
 'State': ['LOCATION', 0.452],
 'Zip': ['ZIPCODE', 1.0],
 'Location (Center point of the Zip Code)': ['ZIPCODE', 1.0]}

#### Contact list Austin

In [126]:
ner("dataset/contact_list_Austin.csv")

Starting...


{'Department': None,
 'First Name': ['PERSON', 0.7777777777777778],
 'Last Name': ['PERSON', 0.4027777777777778],
 'Primary Contact': None,
 'E-Mail Address': ['EMAIL_ADDRESS', 0.8888888888888888],
 'Phone Number': ['PHONE_NUMBER', 0.8611111111111112],
 'Role': None}

#### full-foia-contacts

In [193]:
df_full_noia_contacts = pd.read_csv("dataset/full-foia-contacts.csv")
df_full_noia_contacts

,Agency,Department,Name,Title,Room Number,Street Address,City,State,Zip Code,Telephone,Fax,Email Address,Website,Online Request Form,Notes
0,Agricultural Marketing Service,Department of Agriculture,Gregory Bridges,FOIA Officer,"AG Stop 0202, Room 3521-S","1400 Independence Avenue, SW",Washington,DC,20250-0273,(202) 720-2498,(202) 690-3767,AMS.FOIA@USDA.gov,http://www.ams.usda.gov/about-ams/foia,NaN,NaN
1,Agricultural Marketing Service,Department of Agriculture,NaN,FOIA Requester Service Center,NaN,NaN,NaN,NaN,NaN,(202) 720-2498,NaN,NaN,NaN,NaN,NaN
2,Agricultural Marketing Service,Department of Agriculture,William Allen,FOIA Public Liaison,"AG Stop 0202, Room 3521-S","1400 Independence Avenue, SW",Washington,DC,20250-0273,(202) 720-2498,NaN,AMS.FOIA@USDA.gov,NaN,NaN,NaN
3,Animal & Plant Health Inspection,Department of Agriculture,Tonya Woods,FOIA Director,Unit 50,4700 River Road,Riverdale,MD,20737-1232,(301) 851-4102,(301) 734-5941,mailto:tonya.g.woods@aphis.usda.gov,http://www.aphis.usda.gov/wps/portal/aphis/res...,http://www.aphis.usda.gov/wps/portal/aphis/res...,NaN
4,Animal & Plant Health Inspection,Department of Agriculture,NaN,FOIA Requester Service Center,NaN,NaN,NaN,NaN,NaN,(301) 851-4102,(301) 734-5941,mailto:foia.officer@aphis.usda.gov,NaN,NaN,* Please mail requests to FOIA.Officer@aphis.u...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,Office of Small and Disadvantaged Business Uti...,Department of Veterans Affairs,NaN,FOIA Requester Service Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
743,Office of Small and Disadvantaged Business Uti...,Department of Veterans Affairs,James Horan,FOIA Public Liaison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
744,I don't know which office,Department of Veterans Affairs,Laurie Karnay,FOIA Team Lead,(005R1C) VACO,"810 Vermont Avenue, NW",Washington,DC,20420,(202) 632-7465,(202) 632-7581,NaN,http://www.foia.va.gov/,NaN,NaN
745,I don't know which office,Department of Veterans Affairs,NaN,FOIA Requester Service Center,NaN,NaN,NaN,NaN,NaN,(202) 632-7600,NaN,NaN,NaN,NaN,NaN


In [194]:
ner("dataset/full-foia-contacts.csv")

Starting...


{'Agency': None,
 'Department': None,
 'Name': ['PERSON', 0.643908969210174],
 'Title': ['PERSON', 0.3386880856760375],
 'Room Number': None,
 'Street Address': ['ADDRESS', 0.2744310575635877],
 'City': ['LOCATION', 0.3386880856760375],
 'State': ['LOCATION', 0.26238286479250333],
 'Zip Code': ['ZIPCODE', 1.0],
 'Telephone': ['PHONE_NUMBER', 0.891566265060241],
 'Fax': ['PHONE_NUMBER', 0.3493975903614458],
 'Email Address': ['EMAIL_ADDRESS', 0.4029451137884873],
 'Website': ['URL', 0.34002677376171353],
 'Online Request Form': ['URL', 0.09906291834002677],
 'Notes': None}

In [218]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [226]:
addresses = ['Street', 'Rue', 'Via', 'Square', 'Avenue', 'Place', 'Strada', 'St', 'Lane', 
'Road', 'Boulevard', 'Ln', "Rd", "Highway" "Drive", "Av", "Hwy", "Blvd", "Corso", "Piazza", 
"Calle", "Plaza", "Avenida", "Rambla", "C/"]
addresses_recognizer = PatternRecognizer(supported_entity="ADDRESS", deny_list=addresses)


In [227]:
LANGUAGES_CONFIG_FILE = "/Documents/clearbox_repo/clearbox-ner/model.yml"

# Create NLP engine based on configuration file
provider = NlpEngineProvider(conf_file=LANGUAGES_CONFIG_FILE)
nlp_engine_transformers = provider.create_engine()

# Pass created NLP engine and supported_languages to the AnalyzerEngine
analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine_transformers, 
    supported_languages=["en"]
)

In [228]:
analyzer.registry.add_recognizer(addresses_recognizer)

In [229]:
batch_analyzer = BatchAnalyzerEngine(analyzer_engine=analyzer)

In [231]:
analyzer_results = list(batch_analyzer.analyze_dict(df_full_noia_contacts.to_dict(orient="list"), language="en"))

In [232]:
analyzer_results

[DictAnalyzerResult(key='Agency', value=['Agricultural Marketing Service', 'Agricultural Marketing Service', 'Agricultural Marketing Service', 'Animal & Plant Health Inspection', 'Animal & Plant Health Inspection', 'Animal & Plant Health Inspection', 'Departmental Management (OSEC/OCIO/ FOIA Service Center)', 'Departmental Management (OSEC/OCIO/ FOIA Service Center)', 'Departmental Management (OSEC/OCIO/ FOIA Service Center)', 'Farm Service Agency', 'Farm Service Agency', 'Farm Service Agency', 'Food, Nutrition & Consumer Services', 'Food, Nutrition & Consumer Services', 'Food, Nutrition & Consumer Services', 'Food Safety & Inspection Service', 'Food Safety & Inspection Service', 'Food Safety & Inspection Service', 'Foreign Agricultural Service ', 'Foreign Agricultural Service ', 'Foreign Agricultural Service ', 'Forest Service - Headquarters', 'Forest Service - Headquarters', 'Forest Service - Headquarters', 'Grain Inspection, Packers & Stockyards Administration', 'Grain Inspection, P